# VLM2Vec MMEB-V2 Multi-Vector Benchmark

This Colab notebook provisions an A100 40 GB workflow that installs pinned dependencies, clones the repo, materialises retriever modules, and evaluates vidore/colqwen2.5-v0.2, nvidia/llama-nemoretriever-colembed-3b-v1, and nomic-ai/colnomic-embed-multimodal-3b on MMEB-V2.

## Runtime checklist
- Select **GPU** runtime with **A100 40 GB**.
- Ensure Google Drive access for persistent caches, logs, metrics, and resume state.
- Run cells sequentially; smoke mode validates setup before running the full benchmark.

In [ ]:
import json, os, subprocess
from pathlib import Path

GPU_QUERY = ["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"]
try:
    gpu_info = subprocess.check_output(GPU_QUERY).decode("utf-8").strip().splitlines()
except Exception as exc:
    gpu_info = []
    print(f"nvidia-smi unavailable: {exc}")

print('Detected accelerators:')
for line in gpu_info:
    print(' -', line)

state = {"gpus": gpu_info, "is_a100": any('A100' in line and ('40' in line or '40960' in line) for line in gpu_info)}
Path('/content/work').mkdir(parents=True, exist_ok=True)
with open('/content/work/gpu_detection.json', 'w') as fp:
    json.dump(state, fp, indent=2)

if not state['gpus']:
    raise SystemExit('No NVIDIA GPU detected. Switch to an A100 runtime.')

if state['is_a100']:
    print('✅ NVIDIA A100 40 GB confirmed.')
else:
    print('⚠️ Running without A100. Switch to quick smoke preset and reduce batch sizes.')
    print(json.dumps({"device": "L4 24 GB", "adjustments": {"max_batch_queries": 2, "max_batch_docs": 3, "patch_budget": 768}}, indent=2))

In [ ]:
%%capture install_log
%pip install -U pip==24.2
%pip install torch==2.2.1+cu121 torchvision==0.17.1+cu121 torchaudio==2.2.1+cu121 --index-url https://download.pytorch.org/whl/cu121
%pip install transformers==4.44.0 accelerate==0.33.0 pillow==10.3.0 tqdm==4.66.5 numpy==1.26.4 pyyaml==6.0.2 datasets==2.20.0 huggingface-hub==0.23.4 sentencepiece==0.2.0 safetensors==0.4.3 einops==0.8.0 timm==1.0.7 pandas==2.2.2 rich==13.7.1 keybert==0.7.0 umap-learn==0.5.5 hdbscan==0.8.33 bertopic==0.16.0 evaluate==0.4.1 pynvml==11.5.0 pytest==8.3.2 pytest-cov==4.1.0

In [ ]:
import logging, torch, sys
from rich.console import Console

console = Console(record=True)
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(name)s | %(message)s')

torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

console.print(f'[bold green]Torch device[/bold green]: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 
}')

In [ ]:
from google.colab import drive
from datetime import datetime

drive.mount('/content/drive', force_remount=True)
BASE = Path('/content/work')
PERSIST = Path('/content/drive/MyDrive/vlm2vec')
for path in [BASE, PERSIST, PERSIST / 'outputs', PERSIST / 'cache', PERSIST / 'logs', PERSIST / 'profiler']:
    path.mkdir(parents=True, exist_ok=True)
print('Workspace ready:', BASE)

In [ ]:
%%bash
set -e
cd /content/work
rm -rf VLM2Vec
git clone https://github.com/itsloganmann/VLM2Vec.git
pip install -e VLM2Vec

## Materialise project assets
The following cells copy repository configs, retriever modules, evaluation harness, and tests into `/content/work` so they can be executed and cached within the Colab runtime.

In [ ]:
import shutil
SRC = Path('/content/work/VLM2Vec')
DST = Path('/content/work/runtime')
if DST.exists():
    shutil.rmtree(DST)
shutil.copytree(SRC, DST)
print('Copied project to', DST)

In [ ]:
import subprocess
subprocess.run(['pytest'], cwd='/content/work/runtime', check=True)

In [ ]:
!python /content/work/runtime/evaluation/multi_vector_eval.py --config /content/work/runtime/configs/mmeb_quick_smoke.yaml

## Full evaluation
Run the full MMEB-V2 benchmark after the smoke test passes. Ensure the runtime remains connected (expect several hours on A100).

```python
!python /content/work/runtime/evaluation/multi_vector_eval.py --config /content/work/runtime/configs/mmeb_a100_full.yaml
```